In [1]:
import heapq
import os
import re

import aocd
import numpy as np
from IPython.display import HTML
from scipy.ndimage import convolve

In [2]:
p = aocd.get_puzzle(year=2025, day=4)

In [3]:
desc = "".join(p._get_prose().split("---")[1:])
HTML(desc);

In [4]:
p.examples[0].input_data.split("\n")

['..@@.@@@@.',
 '@@@.@.@.@@',
 '@@@@@.@.@@',
 '@.@@@@..@.',
 '@@.@@@@.@@',
 '.@@@@@@@.@',
 '.@.@.@.@@@',
 '@.@@@.@@@@',
 '.@@@@@@@@.',
 '@.@.@@@.@.']

In [5]:
def get_data(test_data: bool = False):
    if test_data:
        data = p.examples[0].input_data
    else:
        data = p.input_data

    data = data.split("\n")
    return np.array([[c for c in line] for line in data])

## Trad looping approach

In [6]:
data = get_data()

In [7]:
directions = [(1, 0), (0, 1), (-1, 0), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]

In [8]:
def get_pos_char(data, pos):
    if pos[0] < 0 or pos[0] >= data.shape[0]:
        return None
    if pos[1] < 0 or pos[1] >= data.shape[1]:
        return None
    return data[pos[0], pos[1]]

In [9]:
%%time
# Part 1

res = 0

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i, j] == "@":
            found = 0
            for d in directions:
                ni, nj = i + d[0], j + d[1]
                nc = get_pos_char(data, (ni, nj))
                if nc == "@":
                    found += 1
            if found < 4:
                res += 1

res

CPU times: user 51.6 ms, sys: 887 μs, total: 52.5 ms
Wall time: 52 ms


1493

In [10]:
%%time
# Part 2

res = 0

round_res = 1

while round_res > 0:
    new_data = data.copy()
    round_res = 0

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i, j] == "@":
                found = 0
                for d in directions:
                    ni, nj = i + d[0], j + d[1]
                    nc = get_pos_char(data, (ni, nj))
                    if nc == "@":
                        found += 1
                if found < 4:
                    round_res += 1
                    new_data[i, j] = "."
    res += round_res
    data = new_data

res

CPU times: user 1.02 s, sys: 2.75 ms, total: 1.02 s
Wall time: 1.02 s


9194

## Optimised convolution approach

In [11]:
kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

data = get_data()

In [12]:
%%time
# Part 1

mask = (data == "@").astype(int)
neighbor_count = convolve(mask, kernel, mode="constant", cval=0)

res = np.sum((mask == 1) & (neighbor_count < 4))
res

CPU times: user 752 μs, sys: 609 μs, total: 1.36 ms
Wall time: 834 μs


1493

In [13]:
%%time
# Part 2

mask = data == "@"  # Work with boolean array directly

res = 0
round_res = 1

while round_res > 0:
    neighbor_count = convolve(mask.astype(int), kernel, mode="constant", cval=0)
    to_remove = mask & (neighbor_count < 4)
    round_res = np.sum(to_remove)

    mask[to_remove] = False
    res += round_res

res

CPU times: user 7.23 ms, sys: 852 μs, total: 8.08 ms
Wall time: 7.22 ms


9194